In [ ]:
# Iterator
import os
from dateutil import rrule
from datetime import datetime, timedelta
import re
import pprint
# Pandas
import numpy as np
import pandas as pd
# Spotiy Id
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from glob import glob

target = 'spotify_analysis_api'
client_credentials_manager = SpotifyClientCredentials(client_id='e3cddf5da81f43c3a33814866a8de8ed', client_secret='f885f6255fb34c90b8679817d9c63c25')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
destination_dir = '../data/interim/'
iter_name = 'hot100_songs.csv' 
source = pd.read_csv(destination_dir + iter_name,sep='\t').drop_duplicates(subset=['artist','title'])
source['track_artist'] = source['title'] + ' ' + source['artist']

columns = ('iterator','filename','artist', 'title','spotify_artist','spotify_title','popularity','verified')
target_path = './data/iterators/'+ target +'.csv'
try:
    target_df = pd.read_csv(target_path,sep='\t')
except:
    target_df = pd.DataFrame(columns=columns)
target_df['track_artist'] = target_df['title'] + ' ' + target_df['artist']

# iterator = source.loc[~source['track_artist'].isin(target_df['track_artist']) ,:]
iterator = target_df
target_df.drop('track_artist',inplace=True,axis=1)

iterator['spotify_id']
print('Remaining: ', len(iterator))

iteration_count = 1
track_ids = []
for i,(track,artist) in enumerate(iterator['spotify_id']):
    track_ids.append()
    q = track.lower() + '              ' + artist.lower()
    tracks = sp.search(q=q, type='track',market='US')
    if len(tracks['tracks']['items']) == 0:
        q = q.replace('featuring',' ')
        q = q.replace('\"','')
        q = re.sub('([.|\.]*)','', q)
        tracks = sp.search(q=q, type='track',market='US')
        if len(tracks['tracks']['items']) == 0:
            q = q.replace('part iii','part 3')
            q = q.replace('part ii','part 2')
            q = q.replace('part i','part 1')
            q = q.replace('the','')
            q = re.sub('\(.+?\)','', q)
            tracks = sp.search(q=q, type='track')
    try:
        items = tracks['tracks']['items']
        spotify_id = items[0]['id']
        popularity = items[0]['popularity']
        spotify_title = items[0]['name']
        spotify_artist = ' & '.join(list([artist['name'] for artist in items[0]['album']['artists']]))
        verified = (track == spotify_title) and (artist == spotify_artist)
        track_row = pd.DataFrame({'iterator':[spotify_id] ,'filename':[iter_name] ,'artist':[artist] , 'title':[track] ,'spotify_artist':[spotify_artist] ,'spotify_title':[spotify_title] ,'popularity': [popularity],verified':[verified]})
    except:
        print(q)
        print(tracks['tracks']['items'])
        print('Track couldn\'t be found.')
        track_row = pd.DataFrame({'iterator':np.nan ,'filename':[iter_name] ,'artist':[artist] , 'title':[track] ,'spotify_artist': np.nan ,'spotify_title': np.nan ,'spotify_title':[spotify_title] ,'popularity': np.nan,'verified': np.nan})
    target_df = target_df.append(track_row)

    if iteration_count > 9:
        target_df.to_csv(target_path, sep='\t', index=False, encoding='utf-8')
        iteration_count = 1
        time.sleep(1)
    else:
        iteration_count = iteration_count + 1
        time.sleep(.3)
print('no wait')
return target_df, False

In [51]:
# Iterator
import os
from dateutil import rrule
from datetime import datetime, timedelta
import re
import pprint
# Pandas
import numpy as np
import pandas as pd
# Spotiy Id
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from glob import glob

target = 'spotify_ids_hot100'
client_credentials_manager = SpotifyClientCredentials(client_id='e3cddf5da81f43c3a33814866a8de8ed', client_secret='f885f6255fb34c90b8679817d9c63c25')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
destination_dir = '../data/interim/'
iter_name = 'hot100_songs.csv' 
source = pd.read_csv(destination_dir + iter_name,sep='\t').drop_duplicates(subset=['artist','title'])
source['track_artist'] = source['title'] + ' ' + source['artist']

columns = ('iterator','filename','artist', 'title','spotify_artist','spotify_title','popularity','verified')
target_path = '../data/iterators/'+ target +'.csv'
try:
    target_df = pd.read_csv(target_path,sep='\t')
except:
    target_df = pd.DataFrame(columns=columns)
target_df['track_artist'] = target_df['title'] + ' ' + target_df['artist']

# iterator = source.loc[~source['track_artist'].isin(target_df['track_artist']) ,:]
iterator = target_df.dropna(subset=['iterator'])
target_df.drop('track_artist',inplace=True,axis=1)

for i in range(int(len(iterator)/50) + 1):
    start = i * 50
    end = start + 50
    track_ids = list(iterator['iterator'][start:end])
    results = sp.tracks(track_ids)
    tracks = []
    for item in results['tracks']:
        iterator.loc[iterator['iterator'] == item['id'],'popularity'] = item['popularity']
    if i%10 == 0:
        iterator.to_csv(target_path[:-4] + '_test.csv', sep='\t', index=False, encoding='utf-8')
    print(i)

/Users/sebastian/anaconda3/envs/hit_predictor/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/sebastian/anaconda3/envs/hit_predictor/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [53]:
iterator.to_csv(target_path, sep='\t', index=False, encoding='utf-8')